# Quantum Tomography Overview

* **Author:** Gadi Aleksandrowicz (gadia@il.ibm.com)
* **Last Updated:** January 24, 2019



In [1]:
import numpy as np

## The General Theory

The goal of quantum tomography is obtain data on quantum systems by performing several different measurements. In Qiskit Ingis we are currently concerened with the following two tomography tasks:

1. **Quantum state tomograhpy**: Given a quantum state described by a density operator $\rho$, obtain a matrix representation for $\rho$.
2. **Quantum process tomograhpy**: Given a quantum channel $\mathcal{E}$, obtain a matrix representation (the **Choi matrix**) for $\mathcal{E}$.

In both cases we rely on the assumption that we have access to a large number of identical copies of the system and so can peform several different measures on it.

We can roughly split the tomography process to three stages:
1. Preperation: Add suitable initialization/measurement devices to the quantum system.
2. Experiment: Obtain measurement data from the quantum system.
3. Tomography: Use the obtained data to reconstruct the system's description.

Steps 1 and 2 are related to the quantum system being studied, whereas step 3 is a classical computation which can be carried out on standard computers.

## State Tomography Overview

Let $\rho$ be the density operator of a quantum state. If $\rho$ can be reliably reproduced, it can be subjected to several measurements with respect to different operators; this data can be used to reconstruct $\rho$ or a close approximation for it in several different methods.

### Example: 1-qubit reconstruction using the Pauli basis

Given the Pauli matrices $
I=\left(\begin{array}{cc}
1 & 0\\
0 & 1
\end{array}\right),
X=\left(\begin{array}{cc}
0 & 1\\
1 & 0
\end{array}\right),
Y=\left(\begin{array}{cc}
0 & -i\\
i & 0
\end{array}\right),
Z=\left(\begin{array}{cc}
1 & 0\\
0 & -1
\end{array}\right)$


In [3]:
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

It is easy to see they constitute an orthonormal basis for $M_2(\mathbb{C})$ with respect to the Hilbert-Schmidt inner product $\left\langle A,B\right\rangle =\frac{1}{2}\text{tr}\left(B^{\dagger}A\right)$

In [4]:
def HS_product(A,B):
    return np.trace(np.conj(B).T @ A)


And hence,

$$ \rho =\left\langle \rho,I\right\rangle I+\left\langle \rho,X\right\rangle X+\left\langle \rho,Y\right\rangle Y+\left\langle \rho,Z\right\rangle Z = $$

$$=\frac{\text{tr}\left(\rho\right)I+\text{tr}\left(X\rho\right)X+\text{tr}\left(Y\rho\right)Y+\text{tr}\left(Z\rho\right)Z}{2}$$

The values of $\text{tr}\left(X\rho\right), \text{tr}\left(Y\rho\right), \text{tr}\left(Z\rho\right)$ can be approximated by repeated measuring in the $X, Y$ and $Z$ bases. Since $\text{tr}\left(\rho\right)=1$ there is no need for additional measurements for the coefficient of $I$.

### Example: 1-qubit Linear inversion

The above method can be rephrased in more general form. First, any hermitian operator $H$ has a spectral decomposition of the form $H=\sum \lambda_i P_i$ where $\lambda_i$ is an eigenvalue of $H$ and $P_i$ is the projection operator to the corresponding eigenspace. For the hermitian operators $X,Y,Z$ whose eigenvalues are 1 and -1 we can therefore write

* $X = P^X_0-P^X_1$
* $Y = P^Y_0-P^Y_1$
* $Z = P^Z_0-P^Z_1$

Where



$$P^X_0=\frac{1}{2}\left(\begin{array}{cc}
1 & 1\\
1 & 1
\end{array}\right), P^X_1=\frac{1}{2}\left(\begin{array}{cc}
1 & -1\\
-1 & 1
\end{array}\right)$$

$$P^Y_0=\frac{1}{2}\left(\begin{array}{cc}
1 & -i\\
i & 1
\end{array}\right), P^Y_1=\frac{1}{2}\left(\begin{array}{cc}
1 & i\\
-i & 1
\end{array}\right)$$

$$P^Z_0=\left(\begin{array}{cc}
1 & 0\\
0 & 0
\end{array}\right), P^Z_1=\left(\begin{array}{cc}
0 & 0\\
0 & 1
\end{array}\right)$$

In [14]:
PX0 = 0.5*np.array([[1, 1], [1, 1]])
PX1 = 0.5*np.array([[1, -1], [-1, 1]])

PY0 = 0.5*np.array([[1, -1j], [1j, 1]])
PY1 = 0.5*np.array([[1, 1j], [-1j, 1]])

PZ0 = np.array([[1, 0], [0, 0]])
PZ1 = np.array([[0, 0], [0, 1]])

By Born's rule, $\text{tr}\left(P_{i}^{X}\rho\right)$ is the probability for the outcome $\left|i\right\rangle$ when measuring in the X-basis, and this probability can be estimated directly using repeated meausrements in the X-basis. The $Y$ and $Z$ bases are handled
similarily.

The computation $\text{tr}\left(P_{i}^{X}\rho\right)$ can be replaced by the scalar product $\vec{P}_i^X \cdot \vec{\rho}$ where $\vec{E}$ denotes the vector obtained from the operator $E$ by flattening its matrix (the result vector consists of the first row, then the second row etc.)

Now we can construct a matrix $$M=\left(\begin{array}{c}
\vec{P}_{0}^{X}\\
\vec{P}_{1}^{X}\\
\vec{P}_{0}^{Y}\\
\vec{P}_{1}^{Y}\\
\vec{P}_{0}^{Z}\\
\vec{P}_{1}^{Z}
\end{array}\right)$$

Such that $$M\vec{\rho}=\vec{p}=\left(\begin{array}{c}
p_{\left|0\right\rangle }^{X}\\
p_{\left|1\right\rangle }^{X}\\
p_{\left|0\right\rangle }^{Y}\\
p_{\left|1\right\rangle }^{Y}\\
p_{\left|0\right\rangle }^{Z}\\
p_{\left|1\right\rangle }^{Z}
\end{array}\right)$$

Is the equation relating the density operator to the observed probabilities.

In [20]:
M = np.array([
   PX0.flatten(), PX1.flatten(), PY0.flatten(), PY1.flatten(), PZ0.flatten() , PZ1.flatten()
])

Since $M$ can be computed by knowing the operators used in the tomography, and the vector $\vec{p}$ of probabilities can be estimated using the tomography results, all that remains is to solve the equation $M\vec{\rho}=\vec{p}$ for $\vec{\rho}$. If the rank of $M$ is large enough this can be done by multiplying both sides by $M^\dagger$:

$M^\dagger M\vec{\rho} = M^\dagger \vec{p}$

$\vec{\rho} = (M^\dagger M)^{-1} M^\dagger \vec{p}$

In our example, we obtain the matrix 
$$(M^\dagger M)^{-1} M^\dagger = \left(\begin{array}{cccccc}
\frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{4}{6} & -\frac{2}{6}\\
\frac{1}{2} & -\frac{1}{2} & \frac{i}{2} & -\frac{i}{2} & 0 & 0\\
\frac{1}{2} & -\frac{1}{2} & -\frac{i}{2} & \frac{i}{2} & 0 & 0\\
\frac{1}{6} & \frac{1}{6} & \frac{1}{6} & \frac{1}{6} & -\frac{2}{6} & \frac{4}{6}
\end{array}\right)$$

In [27]:
M_dg = np.conj(M).T
linear_inversion_matrix = np.linalg.inv(M_dg @ M) @ M_dg

Multiplication by the linear inversion matrix performs the reconstruction stage described earlier to obtain the density operator.